In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [3]:

# Load the data
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

In [4]:


# Drop columns with non-numeric values
application_df = application_df.drop(columns=['APPLICATION_TYPE', 'STATUS', 'NAME'])

In [5]:

# Convert categorical data to numeric (One-hot encoding for 'INCOME_AMT' column)
application_df = pd.get_dummies(application_df, columns=['INCOME_AMT'])

In [6]:
# Define features and target
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
y = application_df['IS_SUCCESSFUL'].values

In [7]:

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

ValueError: could not convert string to float: 'Independent'

In [ ]:
# Ensure correct data types
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.int32)
X_test = np.array(X_test, dtype=np.float32)
y_test = np.array(y_test, dtype=np.int32)

In [ ]:
# Define the model architecture
def create_model(input_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dropout(0.5),  # Dropout for regularization
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer
    ])
    return model

In [ ]:
# Create the model
model = create_model(input_dim=X_train.shape[1])

In [ ]:
# Define a learning rate schedule
initial_lr = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_lr,
    decay_steps=10000,
    decay_rate=0.9
)

In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Define the early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)


In [ ]:

# Check data types and values
print(np.isnan(X_train).any())  # Should be False
print(np.isinf(X_train).any())  # Should be False
print(np.isnan(y_train).any())  # Should be False
print(np.isinf(y_train).any())  # Should be False

In [ ]:
# Train the model
history = model.fit(
    X_train,  # Features
    y_train,  # Labels
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

In [ ]:
# Save the model
model.save('AlphabetSoupCharity_Optimization.h5')

In [ ]:
# Load the model to verify
loaded_model = tf.keras.models.load_model('AlphabetSoupCharity_Optimization.h5')
loaded_model.summary()
